In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU
import numpy as np
import cv2
import os

model_deplab = load_model("/content/drive/MyDrive/models/model_deeplab.keras", compile=False)
model_unet = load_model("/content/drive/MyDrive/models/model_unet.keras", custom_objects={'LeakyReLU': LeakyReLU}, compile=False)

# Сегментация видео с помощью segnet будет в файле Segnet_Human_Segmentation, потому что в этом файле не получается загрузить

In [ ]:
model_segnet = load_model("/content/drive/MyDrive/models/model_segnet.keras", compile=False)

ValueError: File not found: filepath=/content/drive/MyDrive/models/model_segnet.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
def segment(input_path, output_name, model, background_color=(0, 0, 0)):
    cap = cv2.VideoCapture(input_path)
    frame_width, frame_height, fps = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(os.path.join("/content/drive/MyDrive/videos", f"{output_name}.mp4"), fourcc, fps, (frame_width, frame_height))

    img_size = (256, 256)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # сегментация кадра
        input_img = cv2.resize(frame, img_size) / 255.0 #нормализация
        input_img = np.expand_dims(input_img, axis=0) #добавить измерение
        mask = model.predict(input_img)[0] #предсказание
        mask = cv2.resize(mask, (frame.shape[1], frame.shape[0])) # изменяем размер маски до исходного размера изображения
        _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY) #преобразуем маску в бинарное изображение (0 или 255)
        mask_rgb = np.stack([mask] * 3, axis=-1) #Преобразование маски в ргб формат
        background = np.full_like(frame, background_color) #фоновое изображение с заданным цветом
        segmented_frame = np.where(mask_rgb == 255, frame, background) #Применяем маску к исходному изображению, заменяя пиксели, где маска равна 255, на исходные пиксели, а остальные на фоновый цвет


        out.write(segmented_frame)

    cap.release()
    out.release()



In [50]:
segment("/content/drive/MyDrive/input.mp4", "deplab_video", model_deplab, (0, 0, 0))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 899ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [51]:
segment("/content/drive/MyDrive/input.mp4", "unet_video", model_unet, (0, 0, 0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 